# REDES NEURAIS ARTIFICIAIS

[Aula 3 de Deep Learning](http://bit.ly/dn-unb03) da Engenharia de Software da UnB

In [3]:
# Installing Theano
! pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
! pip install --upgrade tensorflow

# Installing Keras
! pip install --upgrade keras

# Install other packages
! pip install --upgrade pip pandas numpy matplotlib sklearn

  Cloning git://github.com/Theano/Theano.git to /tmp/pip-req-build-q75yrp0e
  Running command git clone -q git://github.com/Theano/Theano.git /tmp/pip-req-build-q75yrp0e
  Created wheel for Theano: filename=Theano-1.0.4+23.g630974a7b-cp37-none-any.whl size=2667499 sha256=60ef67e5a9ecc2bb17af21f82b7d5daf82fd7d1086f97de0529756a51acd36cb
  Stored in directory: /tmp/pip-ephem-wheel-cache-dmpcml90/wheels/ae/32/7c/62beb8371953eb20c271b3bac7d0e56e1a2020d46994346b52
Successfully built Theano
  Found existing installation: Theano 1.0.4+23.g630974a7b
    Uninstalling Theano-1.0.4+23.g630974a7b:
      Successfully uninstalled Theano-1.0.4+23.g630974a7b
Requirement already up-to-date: tensorflow in /home/mrb/Documentos/6_semestre/deeplearning/machinelearning-basics/env/lib/python3.7/site-packages (2.0.0)
     |████████████████████████████████| 583kB 334kB/s eta 0:00:01


  Found existing installation: setuptools 39.0.1
    Uninstalling setuptools-39.0.1:
      Successfully uninstalled setuptools-39.0.1
Requirement already up-to-date: keras in /home/mrb/Documentos/6_semestre/deeplearning/machinelearning-basics/env/lib/python3.7/site-packages (2.3.1)
Requirement already up-to-date: pip in /home/mrb/Documentos/6_semestre/deeplearning/machinelearning-basics/env/lib/python3.7/site-packages (19.3.1)
Requirement already up-to-date: pandas in /home/mrb/Documentos/6_semestre/deeplearning/machinelearning-basics/env/lib/python3.7/site-packages (0.25.3)
Requirement already up-to-date: numpy in /home/mrb/Documentos/6_semestre/deeplearning/machinelearning-basics/env/lib/python3.7/site-packages (1.17.4)
Requirement already up-to-date: matplotlib in /home/mrb/Documentos/6_semestre/deeplearning/machinelearning-basics/env/lib/python3.7/site-packages (3.1.2)
Requirement already up-to-date: sklearn in /home/mrb/Documentos/6_semestre/deeplearning/machinelearning-basics/env

# Parte 1 - Preprocessamento dos Dados

### Importar as libs

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Importar o dataset

In [5]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [7]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]
print("X >>\n",X)

X >>
 [[0.0000000e+00 0.0000000e+00 2.2800000e+02 ... 1.0000000e+00
  1.0000000e+00 1.0134888e+05]
 [1.0000000e+00 0.0000000e+00 2.1700000e+02 ... 0.0000000e+00
  1.0000000e+00 1.1254258e+05]
 [0.0000000e+00 0.0000000e+00 1.1100000e+02 ... 1.0000000e+00
  0.0000000e+00 1.1393157e+05]
 ...
 [0.0000000e+00 0.0000000e+00 3.1800000e+02 ... 0.0000000e+00
  1.0000000e+00 4.2085580e+04]
 [0.0000000e+00 1.0000000e+00 3.8100000e+02 ... 1.0000000e+00
  0.0000000e+00 9.2888520e+04]
 [0.0000000e+00 0.0000000e+00 4.0100000e+02 ... 1.0000000e+00
  0.0000000e+00 3.8190780e+04]]


/home/mrb/Documentos/6_semestre/deeplearning/machinelearning-basics/env/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/mrb/Documentos/6_semestre/deeplearning/machinelearning-basics/env/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

---
# Parte 2 -Vamos construir uma ANN!

## Importando o Keras


In [10]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [11]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Ajustando a ANN ao conjunto de treinamento

In [15]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 20)

Epoch 1/20
8000/8000 [==============================] - 1s 77us/step - loss: 0.3996 - accuracy: 0.8349
Epoch 2/20
8000/8000 [==============================] - 1s 79us/step - loss: 0.3999 - accuracy: 0.8360
Epoch 3/20
8000/8000 [==============================] - 1s 75us/step - loss: 0.3999 - accuracy: 0.8353
Epoch 4/20
8000/8000 [==============================] - 1s 77us/step - loss: 0.3997 - accuracy: 0.8349
Epoch 5/20
8000/8000 [==============================] - 1s 78us/step - loss: 0.3997 - accuracy: 0.8359
Epoch 6/20
8000/8000 [==============================] - 1s 76us/step - loss: 0.3997 - accuracy: 0.8345
Epoch 7/20
8000/8000 [==============================] - 1s 82us/step - loss: 0.3997 - accuracy: 0.8355
Epoch 8/20
8000/8000 [==============================] - 1s 86us/step - loss: 0.3997 - accuracy: 0.8365
Epoch 9/20
8000/8000 [==============================] - 1s 80us/step - loss: 0.3996 - accuracy: 0.8340
Epoch 10/20
8000/8000 [==============================] - 1s 76us/step - l

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [10]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Criando uma Confusion Matrix

In [11]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1510   85]
 [ 196  209]]


# Initiate activation function experimentation

# Initialising the 2nd ANN

In [29]:
del classifier

classifier = Sequential()

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'hard_sigmoid', input_dim = 11))

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'hard_sigmoid'))

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'exponential'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 10, epochs = 20)

Epoch 1/20
8000/8000 [==============================] - 1s 99us/step - loss: 0.6977 - accuracy: 0.6144
Epoch 2/20
8000/8000 [==============================] - 1s 86us/step - loss: 0.4752 - accuracy: 0.7960
Epoch 3/20
8000/8000 [==============================] - 1s 85us/step - loss: 0.4636 - accuracy: 0.7960
Epoch 4/20
8000/8000 [==============================] - 1s 88us/step - loss: 0.4552 - accuracy: 0.7960
Epoch 5/20
8000/8000 [==============================] - 1s 86us/step - loss: 0.4473 - accuracy: 0.7960
Epoch 6/20
8000/8000 [==============================] - 1s 88us/step - loss: 0.4406 - accuracy: 0.7960
Epoch 7/20
8000/8000 [==============================] - 1s 87us/step - loss: 0.4357 - accuracy: 0.7960
Epoch 8/20
8000/8000 [==============================] - 1s 85us/step - loss: 0.4325 - accuracy: 0.7960
Epoch 9/20
8000/8000 [==============================] - 1s 88us/step - loss: 0.4305 - accuracy: 0.7960
Epoch 10/20
8000/8000 [==============================] - 1s 88us/step - l

In [30]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix

In [31]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1521   74]
 [ 243  162]]


# Initialising the 3rd ANN

In [33]:
del classifier

classifier = Sequential()

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh', input_dim = 11))

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid'))

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'tanh'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 10, epochs = 20)

Epoch 1/20
8000/8000 [==============================] - 1s 87us/step - loss: 0.4634 - accuracy: 0.7960
Epoch 2/20
8000/8000 [==============================] - 1s 81us/step - loss: 0.4309 - accuracy: 0.7934
Epoch 3/20
8000/8000 [==============================] - 1s 76us/step - loss: 0.4293 - accuracy: 0.7947
Epoch 4/20
8000/8000 [==============================] - 1s 82us/step - loss: 0.4286 - accuracy: 0.7972
Epoch 5/20
8000/8000 [==============================] - 1s 81us/step - loss: 0.4283 - accuracy: 0.7989
Epoch 6/20
8000/8000 [==============================] - 1s 79us/step - loss: 0.4277 - accuracy: 0.7971
Epoch 7/20
8000/8000 [==============================] - 1s 82us/step - loss: 0.4261 - accuracy: 0.8030
Epoch 8/20
8000/8000 [==============================] - 1s 78us/step - loss: 0.4219 - accuracy: 0.8056
Epoch 9/20
8000/8000 [==============================] - 1s 81us/step - loss: 0.4132 - accuracy: 0.8106
Epoch 10/20
8000/8000 [==============================] - 1s 78us/step - l

In [34]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix

In [35]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1516   79]
 [ 198  207]]


---
# Parte 4 - Evaluating, Improving and Tuning the ANN

## Evaluating the ANN

In [36]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'hard_sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()
print("accuracies:%s\nmean:%s\nvariance:%s".format(accuracies, mean, variance))

accuracies:%s
mean:%s
variance:%s


# Improving the ANN
# Dropout Regularization to reduce overfitting if needed

In [37]:
from keras.layers import Dropout

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'hard_sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

# Tuning the ANN

In [38]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'hard_sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

In [ ]:
parameters = {'batch_size': [10, 24, 32],
              'epochs': [50, 100, 200],
              'optimizer': ['adam', 'rmsprop']}

grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

grid_search = grid_search.fit(X_train, y_train)

best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Epoch 1/50
7200/7200 [==============================] - 1s 115us/step - loss: 0.4944 - accuracy: 0.8031
Epoch 2/50
7200/7200 [==============================] - 1s 94us/step - loss: 0.4420 - accuracy: 0.8083
Epoch 3/50
7200/7200 [==============================] - 1s 98us/step - loss: 0.4376 - accuracy: 0.8106
Epoch 4/50
7200/7200 [==============================] - 1s 110us/step - loss: 0.4370 - accuracy: 0.8085
Epoch 5/50
7200/7200 [==============================] - 1s 101us/step - loss: 0.4370 - accuracy: 0.8072
Epoch 6/50
7200/7200 [==============================] - 1s 98us/step - loss: 0.4352 - accuracy: 0.8097
Epoch 7/50
7200/7200 [==============================] - 1s 101us/step - loss: 0.4343 - accuracy: 0.8086
Epoch 8/50
7200/7200 [==============================] - 1s 99us/step - loss: 0.4251 - accuracy: 0.8142
Epoch 9/50
7200/7200 [==============================] - 1s 104us/step - loss: 0.4200 - accuracy: 0.8186
Epoch 10/50
7200/7200 [==============================] - 1s 112us/st

7200/7200 [==============================] - 1s 92us/step - loss: 0.3965 - accuracy: 0.8369
Epoch 30/50
7200/7200 [==============================] - 1s 93us/step - loss: 0.3892 - accuracy: 0.8374
Epoch 31/50
7200/7200 [==============================] - 1s 89us/step - loss: 0.3926 - accuracy: 0.8356
Epoch 32/50
7200/7200 [==============================] - 1s 93us/step - loss: 0.3954 - accuracy: 0.8364
Epoch 33/50
7200/7200 [==============================] - 1s 91us/step - loss: 0.3912 - accuracy: 0.8415
Epoch 34/50
7200/7200 [==============================] - 1s 91us/step - loss: 0.3896 - accuracy: 0.8386
Epoch 35/50
7200/7200 [==============================] - 1s 93us/step - loss: 0.3940 - accuracy: 0.8365
Epoch 36/50
7200/7200 [==============================] - 1s 89us/step - loss: 0.3931 - accuracy: 0.8386
Epoch 37/50
7200/7200 [==============================] - 1s 94us/step - loss: 0.3896 - accuracy: 0.8403
Epoch 38/50
7200/7200 [==============================] - 1s 89us/step - loss

7200/7200 [==============================] - 1s 92us/step - loss: 0.4243 - accuracy: 0.8201
Epoch 9/50
7200/7200 [==============================] - 1s 89us/step - loss: 0.4249 - accuracy: 0.8211
Epoch 10/50
7200/7200 [==============================] - 1s 92us/step - loss: 0.4182 - accuracy: 0.8253
Epoch 11/50
7200/7200 [==============================] - 1s 89us/step - loss: 0.4148 - accuracy: 0.8264
Epoch 12/50
7200/7200 [==============================] - 1s 91us/step - loss: 0.4068 - accuracy: 0.8293
Epoch 13/50
7200/7200 [==============================] - 1s 93us/step - loss: 0.4041 - accuracy: 0.8338
Epoch 14/50
7200/7200 [==============================] - 1s 88us/step - loss: 0.4004 - accuracy: 0.8339
Epoch 15/50
7200/7200 [==============================] - 1s 92us/step - loss: 0.3946 - accuracy: 0.8379
Epoch 16/50
7200/7200 [==============================] - 1s 89us/step - loss: 0.3994 - accuracy: 0.8364
Epoch 17/50
7200/7200 [==============================] - 1s 91us/step - loss:

Epoch 37/50
7200/7200 [==============================] - 1s 98us/step - loss: 0.3955 - accuracy: 0.8417
Epoch 38/50
7200/7200 [==============================] - 1s 96us/step - loss: 0.3917 - accuracy: 0.8407
Epoch 39/50
7200/7200 [==============================] - 1s 102us/step - loss: 0.3933 - accuracy: 0.8413
Epoch 40/50
7200/7200 [==============================] - 1s 94us/step - loss: 0.3915 - accuracy: 0.8444
Epoch 41/50
7200/7200 [==============================] - 1s 92us/step - loss: 0.3888 - accuracy: 0.8419
Epoch 42/50
7200/7200 [==============================] - 1s 93us/step - loss: 0.3916 - accuracy: 0.8413
Epoch 43/50
7200/7200 [==============================] - 1s 94us/step - loss: 0.3890 - accuracy: 0.8410
Epoch 44/50
7200/7200 [==============================] - 1s 91us/step - loss: 0.3862 - accuracy: 0.8447
Epoch 45/50
7200/7200 [==============================] - 1s 94us/step - loss: 0.3888 - accuracy: 0.8404
Epoch 46/50
7200/7200 [==============================] - 1s 94u

7200/7200 [==============================] - 1s 89us/step - loss: 0.4021 - accuracy: 0.8324
Epoch 16/50
7200/7200 [==============================] - 1s 96us/step - loss: 0.3988 - accuracy: 0.8363
Epoch 17/50
7200/7200 [==============================] - 1s 87us/step - loss: 0.3967 - accuracy: 0.8340
Epoch 18/50
7200/7200 [==============================] - 1s 91us/step - loss: 0.3990 - accuracy: 0.8351
Epoch 19/50
7200/7200 [==============================] - 1s 103us/step - loss: 0.3982 - accuracy: 0.8392
Epoch 20/50
7200/7200 [==============================] - 1s 108us/step - loss: 0.4022 - accuracy: 0.8347
Epoch 21/50
7200/7200 [==============================] - 1s 143us/step - loss: 0.3947 - accuracy: 0.8417
Epoch 22/50
7200/7200 [==============================] - 1s 112us/step - loss: 0.3964 - accuracy: 0.8354
Epoch 23/50
7200/7200 [==============================] - 1s 100us/step - loss: 0.3975 - accuracy: 0.8339
Epoch 24/50
7200/7200 [==============================] - 1s 98us/step -

...

7200/7200 [==============================] - 1s 92us/step - loss: 0.4966 - accuracy: 0.8290
Epoch 28/50
7200/7200 [==============================] - 1s 86us/step - loss: 0.4948 - accuracy: 0.8301
Epoch 29/50
7200/7200 [==============================] - 1s 94us/step - loss: 0.5072 - accuracy: 0.8282
Epoch 30/50
7200/7200 [==============================] - 1s 93us/step - loss: 0.4730 - accuracy: 0.8292
Epoch 31/50
7200/7200 [==============================] - 1s 104us/step - loss: 0.4917 - accuracy: 0.8328
Epoch 32/50
7200/7200 [==============================] - 1s 94us/step - loss: 0.5014 - accuracy: 0.8363
Epoch 33/50
7200/7200 [==============================] - 1s 91us/step - loss: 0.4984 - accuracy: 0.8315
Epoch 34/50
7200/7200 [==============================] - 1s 104us/step - loss: 0.4958 - accuracy: 0.8292
Epoch 35/50
7200/7200 [==============================] - 1s 95us/step - loss: 0.4896 - accuracy: 0.8329
Epoch 36/50
7200/7200 [==============================] - 1s 90us/step - lo

7200/7200 [==============================] - 1s 82us/step - loss: 0.4756 - accuracy: 0.8026
Epoch 6/50
7200/7200 [==============================] - 1s 84us/step - loss: 0.4557 - accuracy: 0.8053
Epoch 7/50
7200/7200 [==============================] - 1s 82us/step - loss: 0.4697 - accuracy: 0.8018
Epoch 8/50
7200/7200 [==============================] - 1s 84us/step - loss: 0.4617 - accuracy: 0.8026
Epoch 9/50
7200/7200 [==============================] - 1s 82us/step - loss: 0.4593 - accuracy: 0.8026
Epoch 10/50
7200/7200 [==============================] - 1s 85us/step - loss: 0.4617 - accuracy: 0.8064
Epoch 11/50
7200/7200 [==============================] - 1s 83us/step - loss: 0.4718 - accuracy: 0.8053
Epoch 12/50
7200/7200 [==============================] - 1s 85us/step - loss: 0.4667 - accuracy: 0.8071
Epoch 13/50
7200/7200 [==============================] - 1s 87us/step - loss: 0.5173 - accuracy: 0.8083
Epoch 14/50
7200/7200 [==============================] - 1s 85us/step - loss: 0.

7200/7200 [==============================] - 1s 84us/step - loss: 0.4807 - accuracy: 0.8324
Epoch 34/50
7200/7200 [==============================] - 1s 82us/step - loss: 0.4801 - accuracy: 0.8322
Epoch 35/50
7200/7200 [==============================] - 1s 84us/step - loss: 0.4653 - accuracy: 0.8299
Epoch 36/50
7200/7200 [==============================] - 1s 83us/step - loss: 0.4844 - accuracy: 0.8308
Epoch 37/50
7200/7200 [==============================] - 1s 85us/step - loss: 0.4751 - accuracy: 0.8346
Epoch 38/50
7200/7200 [==============================] - 1s 82us/step - loss: 0.4829 - accuracy: 0.8336
Epoch 39/50
7200/7200 [==============================] - 1s 85us/step - loss: 0.5087 - accuracy: 0.8342
Epoch 40/50
7200/7200 [==============================] - 1s 82us/step - loss: 0.5020 - accuracy: 0.8353
Epoch 41/50
7200/7200 [==============================] - 1s 84us/step - loss: 0.5268 - accuracy: 0.8353
Epoch 42/50
7200/7200 [==============================] - 1s 83us/step - loss

7200/7200 [==============================] - 1s 178us/step - loss: 0.4553 - accuracy: 0.8172
Epoch 12/50
7200/7200 [==============================] - 1s 110us/step - loss: 0.4568 - accuracy: 0.8208
Epoch 13/50
7200/7200 [==============================] - 1s 94us/step - loss: 0.4551 - accuracy: 0.8232
Epoch 14/50
7200/7200 [==============================] - 1s 112us/step - loss: 0.5022 - accuracy: 0.8256
Epoch 15/50
7200/7200 [==============================] - 1s 119us/step - loss: 0.4849 - accuracy: 0.8263
Epoch 16/50
7200/7200 [==============================] - 1s 83us/step - loss: 0.4467 - accuracy: 0.8289
Epoch 17/50
7200/7200 [==============================] - 1s 86us/step - loss: 0.4655 - accuracy: 0.8296
Epoch 18/50
7200/7200 [==============================] - 1s 111us/step - loss: 0.4740 - accuracy: 0.8296
Epoch 19/50
7200/7200 [==============================] - 1s 87us/step - loss: 0.4690 - accuracy: 0.8360
Epoch 20/50
7200/7200 [==============================] - 1s 107us/step 

7200/7200 [==============================] - 1s 89us/step - loss: 0.5230 - accuracy: 0.8367
Epoch 40/50
7200/7200 [==============================] - 1s 88us/step - loss: 0.5062 - accuracy: 0.8319
Epoch 41/50
7200/7200 [==============================] - 1s 88us/step - loss: 0.4581 - accuracy: 0.8338
Epoch 42/50
7200/7200 [==============================] - 1s 88us/step - loss: 0.4705 - accuracy: 0.8358
Epoch 43/50
7200/7200 [==============================] - 1s 88us/step - loss: 0.4973 - accuracy: 0.8308
Epoch 44/50
7200/7200 [==============================] - 1s 89us/step - loss: 0.5123 - accuracy: 0.8369
Epoch 45/50
7200/7200 [==============================] - 1s 85us/step - loss: 0.4947 - accuracy: 0.8353
Epoch 46/50
7200/7200 [==============================] - 1s 88us/step - loss: 0.5137 - accuracy: 0.8394
Epoch 47/50
7200/7200 [==============================] - 1s 85us/step - loss: 0.4982 - accuracy: 0.8335
Epoch 48/50
7200/7200 [==============================] - 1s 88us/step - loss

# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)